In [ ]:

# Convert Matlab code to Python notebook, place it on Git and share it via Google Colab

import numpy as np
import matplotlib.pyplot as plt
import random

def poissonGenKreuz(nSpikes, T):

    uniform = np.random.rand(1, nSpikes)
    spikes = -np.log(1-uniform)/(nSpikes/T)
    spikes = np.cumsum(spikes)
    spikes = spikes - min(spikes)
    spikes = T*spikes/max(spikes)
    return spikes


def switch(case, spikes, nSpikes, T, jit):
    return {
        'same': spikes[0],
        'different': poissonGenKreuz(nSpikes, T),
        'same with some noise': spikes[0] + jit
    }.get(case, None)


def latency_cross(nBin, noiseLevelj):
    # jitter should be from 0 to 1; fool proof later
    # Determining the latency using spike train metrics
    # started April Fool 2015
    # @Nebojsa @Mario @Thomas d_KreuzLab_b

    nStats = 100
    nSpikes = 1000

    T = 100

    # noiseLevelj = 0.1;
    noiseLeveld = 0
    noiseLevela = 0

    # rate is defined as nSpikes/T

    # modify to a point Spike_distance in order to cover the edge
    tauStep = T / nBin # .01
    Tau = np.arange(-T / 2, T / 2, tauStep)

    T_d = np.zeros([len(Tau)])
    Tc_d = np.zeros([len(Tau)])
    Tv_d = np.zeros([len(Tau)])

    cases = ['same', 'different', 'same with some noise']
    doit = cases[2]

    spikes = np.empty((2,), dtype=object)

    for i in range(nStats):

        # print(i)
        ## spikes{0} = poissonSpikeGen(nSpikes / T, T) omit?@###
        spikes[0] = poissonGenKreuz(nSpikes, T)

        jit = np.sort(np.random.rand(len(spikes[0])))
        if (noiseLevelj):
            jit = noiseLevelj * T / nSpikes * jit / max(jit)
        else:
            jit = np.zeros(len(jit))
        jit[0] = abs(jit[0])
        jit[-1] = -abs(jit[-1])

        ref_spikes = switch(doit, spikes, nSpikes, T, jit)

        C_c = np.zeros(len(Tau))
        S_d = np.zeros(len(Tau))
        V_c = np.zeros(len(Tau))

        for count in range(len(Tau)):
            print(count)
            spikes[1] = ref_spikes + Tau[count]
            # (i-1)/nStats*100 + (count-1)/len(Tau)*100/nStats

            # # save test.mat spikes Tau count ?
            # temp = spkDistance2(spikes, max(0, Tau(count)), min(T, T + Tau(count)), [0, Tau(count)], [T, T + Tau(count)])
            # tauStep = (min(T, T + Tau(count)) - max(0, Tau(count))) / nBin
            # psth[0] = histc(spikes[0], max(0, Tau(count)): tauStep: min(T, T + Tau(count)))
            # psth[1] = histc(spikes[1], max(0, Tau(count)): tauStep: min(T, T + Tau(count)))  # tauStep
            # R = corrcoef(psth[0][:-1), psth[1][:-1])
            # vc = SPIKY_Victor_MEX(spikes[0], spikes[1], 100)  # put as an argument 100

            C_c[count] = 1  # R[0, 1]
            S_d[count] = 2  # temp
            V_c[count] = 3  # vc

        T_d = T_d + S_d  # make a matrix
        Tc_d = Tc_d + C_c  # make a matrix
        Tv_d = Tv_d + V_c  # make a matrix

    normT_d = (T_d/nStats - np.mean(T_d/nStats))/np.std(T_d/nStats)
    normTc_d = -(Tc_d/nStats - np.mean(Tc_d/nStats))/np.std(Tc_d/nStats)
    normTv_d = (Tv_d/nStats - np.mean(Tv_d/nStats))/np.std(Tv_d/nStats)
    # plot(Tau, normT_d, 'k', Tau, normTc_d, 'm')
    # title(str2num(nSpikes / tauStep))
    # size(T_d);
    b = [np.mean(T_d/nStats)- np.mean(Tc_d/nStats), np.std(T_d/nStats), np.std(Tc_d/nStats)]
    b[0] = normT_d[(len(normT_d) + 1) // 2]  # middle value
    b[1] = normTc_d[(len(normTc_d) + 1) // 2]
    b[2] = normTv_d[(len(normTv_d) + 1) // 2]

    return b

# nBin = np.arange(50,500,50) # 10**np.arange(1,3)
nBin = 100
jitter = np.array([0, 0.05, 0.1, 0.2, 0.4, 0.8, 1.6])
spkDistv = np.zeros(np.size(jitter))
crosscorv = np.zeros(np.size(jitter))
vp = np.zeros(np.size(jitter))

for count, val in enumerate(jitter):
    print(count)

    b = latency_cross(nBin, jitter[count])
    spkDistv[count] = b[0]
    crosscorv[count] = b[1]
    vp[count] = b[2]

plt.plot(jitter, spkDistv, jitter, crosscorv, jitter, vp)
#plt.legend(("SPIKE-distance", "Cross-correlation", "Victor Purpura"))
#plt.legend(loc="upper left")
plt.ylabel('Amplitude of the peak')
plt.xlabel('Jitter')
plt.show()